In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
#(Completed)
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
#(Completed)

username = "aacuser"
password = "LincolnMikala!3"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
#(Completed)
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
#(Completed)

app.layout = html.Div([
    html.Div([
        # Client logo encoded in Base64 so it u uploads directly in the Dash
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '60px'}
            ),
        ),
        # Dashboard title
        html.Span("  Grazioso Salvare Dashboard - Freddy Case",
                  style={'fontSize': '24px', 'fontWeight': 'bold', 'marginLeft': '15px'})
    # Align logo and title on one row
    ], style={'display': 'flex', 'alignItems': 'center'}),
#    html.Div(id='hidden-div', style={'display':'none'}),
    # Center dashboard
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
   # html.Div(
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
#(Completed)
        html.Div([
        html.Label('Rescue Type Filter'),
        # Add radio buttons
        dcc.RadioItems(
            id='filter-type',
            # Each type of button
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
             # Default setting loads entire dataset
            value='reset',
            labelStyle={'display': 'inline-block', 'marginRight': '15px'}
        )
    ], style={'textAlign': 'center'}),

    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
##(Completed)
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_rows=[0],
                         selected_columns=[],
                         page_action="native",
                         page_current=0,
                         page_size=10,  
                        ),
    html.Br(),
    html.Hr(),
    # Sets up the dashboard so that the chart and geolocation are side by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    global df
## FIX ME Add code to filter interactive data table with MongoDB queries
#(Completed)
    # Dashboard loads in with no filter
    query = {}

    # Filter for water rescue breeds
    if filter_type == 'water':
        # The data that will be filtered
        query = {
            'animal_type': 'Dog',
            'breed': {'$in': [
                'Labrador Retriever Mix',
                'Chesapeake Bay Retriever',
                'Newfoundland'
            ]},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    # Filter for mountain rescue breeds
    elif filter_type == 'mountain':
        query = {
            'animal_type': 'Dog',
            'breed': {'$in': [
                'German Shepherd',
                'Alaskan Malamute',
                'Old English Sheepdog',
                'Siberian Husky',
                'Rottweiler'
            ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    # Filter for disaster rescue breeds
    elif filter_type == 'disaster':
        query = {
            'animal_type': 'Dog',
            'breed': {'$in': [
                'Doberman Pinscher',
                'German Shepherd',
                'Golden Retriever',
                'Bloodhound',
                'Rottweiler'
            ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    # Use MongoDB with the Crud module
    df = pd.DataFrame.from_records(db.read(query))

    # Removes MongoDB_id to avoid rendering issues
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
###FIX ME ####
# add code for chart of your choice (e.g. pie chart) #
#(Completed)
    global df

    # If the table is rendered, build chart from that data
    if viewData is not None:
        dff = pd.DataFrame(viewData)
        if not dff.empty:
            df = dff

    # Get count of breeds
    breed_counts = df['breed'].value_counts()

    # Display only Top 10 breeds only
    top_10 = breed_counts.head(10)

    # Add all of the remaining breeds together as "Other"
    other_total = breed_counts.iloc[10:].sum()

    # Build a new DataFrame for the pie chart
    chart_df = top_10.reset_index()
    chart_df.columns = ['breed', 'count']

    if other_total > 0:
        chart_df.loc[len(chart_df)] = ['Other', other_total]
    
    # Keep only the top 10 breeds so the pie is readable
    if 'breed' in df.columns and not df.empty:
        top_breeds = df['breed'].value_counts().head(10).index.tolist()
        df = df[df['breed'].isin(top_breeds)]
    
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server(mode='inline', port=8051)